In [1]:
import tensorflow as tf
print(tf.__version__)

2.6.0


## 0. Imports

In [11]:
import pandas as pd
import numpy as np
import os
import logging
from google.cloud.storage import Client

## 1. Config work

In [12]:
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT=PROJECT_ID[0]
BUCKET_NAME = f"{PROJECT}-tweet-sentiment-analysis"
BUCKET= f"gs://{PROJECT}-tweet-sentiment-analysis"
ROOT='sentiment-classifcation'
MODEL_DIR=os.path.join(ROOT,'models').replace("\\","/")
PACKAGES_DIR=os.path.join(ROOT,'packages').replace("\\","/")
REGION = 'europe-west1'

In [13]:
!gcloud config set project {PROJECT}

Updated property [core/project].


In [14]:
!gsutil mb -l {REGION} {BUCKET}

Creating gs://rare-result-248415-tweet-sentiment-analysis/...
ServiceException: 409 A Cloud Storage bucket named 'rare-result-248415-tweet-sentiment-analysis' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [15]:
!gsutil rm -r {BUCKET}/{ROOT}

CommandException: No URLs matched: gs://rare-result-248415-tweet-sentiment-analysis/sentiment-classifcation


In [16]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

## 2. Get data

### 2.1. Input data

In [17]:
sentiment_mapping={
    0:"negative",
    2:"neutral",
    4:"positive"
}

data can be downloaded from: https://www.kaggle.com/kazanova/sentiment140

In [18]:
df_twitter = pd.read_csv(BUCKET+"/Data/sentiment_140/training.csv",encoding="latin1", header=None)\
             .rename(columns={
                 0:"sentiment",
                 1:"id",
                 2:"time",
                 3:"query",
                 4:"username",
                 5:"text"
             })[["sentiment","text"]]

In [19]:
df_twitter["sentiment_label"] = df_twitter["sentiment"].map(sentiment_mapping)

### 2.2. Data processing fn

In [20]:
%%writefile preprocess.py

from tensorflow.python.keras.preprocessing import sequence
from tensorflow.keras.preprocessing import text
import re

class TextPreprocessor(object):
    def _clean_line(self, text):
        text = re.sub(r"http\S+", "", text)
        text = re.sub(r"@[A-Za-z0-9]+", "", text)
        text = re.sub(r"#[A-Za-z0-9]+", "", text)
        text = text.replace("RT","")
        text = text.lower()
        text = text.strip()
        return text
    
    def __init__(self, vocab_size, max_sequence_length):
        self._vocab_size = vocab_size
        self._max_sequence_length = max_sequence_length
        self._tokenizer = None

    def fit(self, text_list):        
        # Create vocabulary from input corpus.
        text_list_cleaned = [self._clean_line(txt) for txt in text_list]
        tokenizer = text.Tokenizer(num_words=self._vocab_size)
        tokenizer.fit_on_texts(text_list)
        self._tokenizer = tokenizer

    def transform(self, text_list):        
        # Transform text to sequence of integers
        text_list = [self._clean_line(txt) for txt in text_list]
        text_sequence = self._tokenizer.texts_to_sequences(text_list)

        # Fix sequence length to max value. Sequences shorter than the length are
        # padded in the beginning and sequences longer are truncated
        # at the beginning.
        padded_text_sequence = sequence.pad_sequences(
          text_sequence, maxlen=self._max_sequence_length)
        return padded_text_sequence

Overwriting preprocess.py


Some small test:

In [21]:
from preprocess import TextPreprocessor

processor = TextPreprocessor(5, 5)
processor.fit(['hello machine learning','test'])
processor.transform(['hello machine learning',"lol"])

array([[0, 0, 1, 2, 3],
       [0, 0, 0, 0, 0]], dtype=int32)

### 2.3. Prep data

In [22]:
CLASSES = {'negative':0, 'positive': 1}  # label-to-int mapping
VOCAB_SIZE = 25000  # Limit on the number vocabulary size used for tokenization
MAX_SEQUENCE_LENGTH = 50  # Sentences will be truncated/padded to this length

In [23]:
from preprocess import TextPreprocessor
from sklearn.model_selection import train_test_split

sents = df_twitter.text
labels = np.array(df_twitter.sentiment_label.map(CLASSES))

# Train and test split
X, _, y, _ = train_test_split(sents, labels, test_size=0.1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Create vocabulary from training corpus.
processor = TextPreprocessor(VOCAB_SIZE, MAX_SEQUENCE_LENGTH)
processor.fit(X_train)

# Preprocess the data
train_texts_vectorized = processor.transform(X_train)
eval_texts_vectorized = processor.transform(X_test)

In [24]:
import pickle

with open('./processor_state.pkl', 'wb') as f:
    pickle.dump(processor, f)

## 3. Model

In [25]:
LEARNING_RATE=.001
EMBEDDING_DIM=50
FILTERS=64
DROPOUT_RATE=0.5
POOL_SIZE=3
NUM_EPOCH=1
BATCH_SIZE=128
KERNEL_SIZES=[2,5,8]

### 3.1. Basic model

In [26]:
def create_model(vocab_size, embedding_dim, filters, kernel_sizes, dropout_rate, pool_size, embedding_matrix):
    
    # Input layer
    model_input = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

    # Embedding layer
    z = tf.keras.layers.Embedding(
        input_dim=vocab_size+1,
        output_dim=embedding_dim,
        input_length=MAX_SEQUENCE_LENGTH,
        weights=[embedding_matrix]
    )(model_input)

    z = tf.keras.layers.Dropout(dropout_rate)(z)

    # Convolutional block
    conv_blocks = []
    for kernel_size in kernel_sizes:
        conv = tf.keras.layers.Convolution1D(
            filters=filters,
            kernel_size=kernel_size,
            padding="valid",
            activation="relu",
            bias_initializer='random_uniform',
            strides=1)(z)
        conv = tf.keras.layers.MaxPooling1D(pool_size=2)(conv)
        conv = tf.keras.layers.Flatten()(conv)
        conv_blocks.append(conv)
        
    z = tf.keras.layers.Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]

    z = tf.keras.layers.Dropout(dropout_rate)(z)
    z = tf.keras.layers.Dense(100, activation="relu")(z)
    model_output = tf.keras.layers.Dense(1, activation="sigmoid")(z)

    model = tf.keras.models.Model(model_input, model_output)
    
    return model

### 3.2. Pretrained Glove embeddings

embedding can be downloaded here: https://nlp.stanford.edu/projects/glove/

In [ ]:
client = Client()
bucket = client.get_bucket(BUCKET_NAME)
temp_folder = "Data/embeddings/"
if not os.path.exists(temp_folder):
    os.makedirs(temp_folder)
blob = bucket.get_blob("Data/embeddings/glove.twitter.27B.50d.txt")
downloaded_file = blob.download_to_filename(temp_folder+'/glove.twitter.27B.50d.txt')

In [27]:
def get_coaefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coaefs(*o.strip().split()) for o in
                                                open(temp_folder+"/glove.twitter.27B.50d.txt","r",encoding="utf8"))

In [28]:
word_index = processor._tokenizer.word_index
nb_words = min(VOCAB_SIZE, len(word_index))
embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))

for word, i in word_index.items():
    if i >= VOCAB_SIZE: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

### 3.3. Create - compile - train

In [29]:
model = create_model(VOCAB_SIZE, EMBEDDING_DIM, FILTERS, KERNEL_SIZES, DROPOUT_RATE,POOL_SIZE, embedding_matrix)

2022-01-27 09:51:17.452998: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-01-27 09:51:17.453048: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-27 09:51:17.453083: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tweet-sentiment-analysis): /proc/driver/nvidia/version does not exist
2022-01-27 09:51:17.453521: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
202

In [30]:
# Compile model with learning parameters.
optimizer = tf.keras.optimizers.Nadam(lr=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])

In [31]:
#keras train
history = model.fit(
    train_texts_vectorized, 
    y_train, 
    epochs=NUM_EPOCH, 
    batch_size=BATCH_SIZE,
    validation_data=(eval_texts_vectorized, y_test),
    verbose=2,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_acc',
            min_delta=0.005,
            patience=3,
            factor=0.5),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            min_delta=0.005, 
            patience=5, 
            verbose=0, 
            mode='auto'
        ),
        tf.keras.callbacks.History()
    ]
)

8438/8438 - 228s - loss: 0.4833 - acc: 0.7664 - val_loss: 0.4259 - val_acc: 0.8043


In [32]:
with open("history.pkl",'wb') as file:
    pickle.dump(history.history,file)

In [33]:
model.save('keras_saved_model.h5')

## 4. Deployment

### 4.1. Prepare custom model prediction

In [34]:
%%writefile model_prediction.py

import os
import pickle
import numpy as np

class CustomModelPrediction(object):

  def __init__(self, model, processor):
    # Class gets instantiated with a trained model file and a persisted processor
    self._model = model
    self._processor = processor

  def _postprocess(self, predictions):
    # Create an output signature
    labels = ['negative', 'positive']
    return [
        {
            "label":labels[int(np.round(prediction))],
            "score":float(np.round(prediction,4))
        } for prediction in predictions]

  def predict(self, instances, **kwargs):
    # Clean the data, make predictions and postprocess
    preprocessed_data = self._processor.transform(instances)
    predictions =  self._model.predict(preprocessed_data)
    labels = self._postprocess(predictions)
    return labels

  @classmethod
  def from_path(cls, model_dir):
    # Load the keras model and the persisted processor
    import tensorflow.keras as keras
    
    model = keras.models.load_model(
      os.path.join(model_dir,'saved_model.pb'))
    
    # I know, pickle is bad and I should feel bad
    with open(os.path.join(model_dir, 'processor_state.pkl'), 'rb') as f:
      processor = pickle.load(f)

    return cls(model, processor)

Overwriting model_prediction.py


Test

In [35]:
requests = (["God I hate the north","god I love this"])

In [36]:
from model_prediction import CustomModelPrediction

classifier = CustomModelPrediction.from_path('.')
results = classifier.predict(requests)
results

[{'label': 'negative', 'score': 0.06350000202655792},
 {'label': 'positive', 'score': 0.9175999760627747}]

### 4.2. Package it

In [37]:
%%writefile setup.py

from setuptools import setup

setup(
  name="tweet_sentiment_classifier",
  version="0.1",
  include_package_data=True,
  scripts=["preprocess.py", "model_prediction.py"]
)

Overwriting setup.py


Wrap it up and copy to GCP

In [38]:
!python setup.py sdist
!gsutil cp ./dist/tweet_sentiment_classifier-0.1.tar.gz {BUCKET}/{PACKAGES_DIR}/tweet_sentiment_classifier-0.1.tar.gz

running sdist
running egg_info
writing tweet_sentiment_classifier.egg-info/PKG-INFO
writing dependency_links to tweet_sentiment_classifier.egg-info/dependency_links.txt
writing top-level names to tweet_sentiment_classifier.egg-info/top_level.txt
reading manifest file 'tweet_sentiment_classifier.egg-info/SOURCES.txt'
writing manifest file 'tweet_sentiment_classifier.egg-info/SOURCES.txt'

running check


creating tweet_sentiment_classifier-0.1
creating tweet_sentiment_classifier-0.1/tweet_sentiment_classifier.egg-info
creating tweet_sentiment_classifier-0.1/tweet_sentiment_classifier.egg-info/.ipynb_checkpoints
copying files to tweet_sentiment_classifier-0.1...
copying model_prediction.py -> tweet_sentiment_classifier-0.1
copying preprocess.py -> tweet_sentiment_classifier-0.1
copying setup.py -> tweet_sentiment_classifier-0.1
copying tweet_sentiment_classifier.egg-info/PKG-INFO -> tweet_sentiment_classifier-0.1/tweet_sentiment_classifier.egg-info
copying tweet_sentiment_classifier.egg-

In [39]:
!gsutil cp keras_saved_model.h5 {BUCKET}/{MODEL_DIR}/
!gsutil cp processor_state.pkl {BUCKET}/{MODEL_DIR}/

Copying file://keras_saved_model.h5 [Content-Type=application/x-hdf5]...
/ [1 files][ 19.9 MiB/ 19.9 MiB]                                                
Operation completed over 1 objects/19.9 MiB.                                     
Copying file://processor_state.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 27.2 MiB/ 27.2 MiB]                                                
Operation completed over 1 objects/27.2 MiB.                                     


## 5. Create model and version

In [40]:
MODEL_NAME='tweet_sentiment_classifier'
VERSION_NAME='V1'
RUNTIME_VERSION='2.3' # tensorflow version
REGION='europe-west1'

In [63]:
!gcloud ai-platform models create {MODEL_NAME} --regions {REGION}

Using endpoint [https://ml.googleapis.com/]
ERROR: (gcloud.ai-platform.models.create) Resource in projects [rare-result-248415] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name


In [64]:
!gcloud ai-platform models list --region global

Using endpoint [https://ml.googleapis.com/]
NAME                        DEFAULT_VERSION_NAME
tweet_sentiment_classifier


In [80]:
!gcloud ai-platform versions delete {VERSION_NAME} --model {MODEL_NAME} --region global --quiet

Using endpoint [https://ml.googleapis.com/]
Deleting version [V1]......done.                                               


In [66]:
!gcloud beta ai-platform versions create {VERSION_NAME} \
--model {MODEL_NAME} \
--origin {BUCKET}/{MODEL_DIR} \
--python-version 3.7 \
--runtime-version {RUNTIME_VERSION} \
--package-uris {BUCKET}/{PACKAGES_DIR}/tweet_sentiment_classifier-0.1.tar.gz \
--prediction-class=model_prediction.CustomModelPrediction \
--region global

Using endpoint [https://ml.googleapis.com/]
Creating version (this might take a few minutes)......done.                    


## 6. Testing

In [72]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

In [73]:
requests = [
    "god this episode sucks",
    "meh, I kinda like it",
    "what were the writer thinking, omg!",
    "omg! what a twist, who would'v though :o!"
    "woohoow, sansa for the win!"
]

In [74]:
# JSON format the requests
request_data = {'instances': requests}

# Authenticate and call CMLE prediction API 
credentials = GoogleCredentials.get_application_default()

In [78]:
%%time

api = discovery.build('ml', 'v1')

parent = 'projects/{}/models/{}/versions/{}'.format(PROJECT, MODEL_NAME, VERSION_NAME)
parent = 'projects/{}/models/{}'.format(PROJECT, MODEL_NAME)
response = api.projects().predict(body=request_data, name=parent).execute()

CPU times: user 15.6 ms, sys: 4.13 ms, total: 19.8 ms
Wall time: 212 ms


In [79]:
response["predictions"]

[{'label': 'negative', 'score': 0.023399999365210533},
 {'label': 'positive', 'score': 0.6355000138282776},
 {'label': 'positive', 'score': 0.6098999977111816},
 {'label': 'positive', 'score': 0.5871000289916992}]